In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [3]:
train.shape

(23814, 876)

In [4]:
test = pd.read_csv("../input/lish-moa/test_features.csv")
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [5]:
train['cp_type'] = (train['cp_type'] == 'trt_cp').astype('int')
train['cp_dose'] = (train['cp_dose'] == 'D1').astype('int')

In [6]:
test['cp_type'] = (test['cp_type'] == 'trt_cp').astype('int')
test['cp_dose'] = (test['cp_dose'] == 'D1').astype('int')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(3), object(1)
memory usage: 159.2+ MB


In [8]:
train_NS = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_NS.head()

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
target = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
target.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
target.shape

(23814, 207)

In [11]:
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')
sub = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [12]:
label_encoder = sklearn.preprocessing.LabelEncoder()

In [13]:
mapped_drug_id = pd.Series(label_encoder.fit_transform(train_drug['drug_id']))
train_drug['drug_id'] = mapped_drug_id
train_drug.head()

,sig_id,drug_id
0,id_000644bb2,2280
1,id_000779bfc,2832
2,id_000a6266a,290
3,id_0015fd391,1719
4,id_001626bd3,1511


In [14]:
train.merge(train_drug)

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,drug_id
0,id_000644bb2,1,24,1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,2280
1,id_000779bfc,1,72,1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,2832
2,id_000a6266a,1,48,1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,290
3,id_0015fd391,1,48,1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,1719
4,id_001626bd3,1,72,0,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,1511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,24,0,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680,2825
23810,id_fffb70c0c,1,24,0,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689,3023
23811,id_fffc1c3f4,0,48,0,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808,2544
23812,id_fffcb9e7c,1,24,1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740,1705


In [15]:
def run_lgbm(target_col: str):
    
    X_train = train.drop(["sig_id"], axis=1) # train без sig_id
    y_train = target[target_col]             # столбец из цикла for ниже
    X_test = test.drop(["sig_id"], axis=1)   # test без sig_id

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))   # [0. 0. 0. ... 0. 0. 0.] длиной 23814

    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

In [16]:
# X_train = train.drop(["sig_id"], axis=1)
# oof_train = np.zeros((len(X_train),))
# print(oof_train)

In [17]:
# from catboost import CatBoostClassifier
# cat = CatBoostClassifier(random_seed=17)

# def run_cat(target_col: str):
    
#     X_train = train.drop(["sig_id"], axis=1) # train без sig_id
#     y_train = target[target_col]             # столбец из цикла for ниже
    
#     y_preds = []
#     models = []
#     oof_train = np.zeros((len(X_train),))
    
#     X_train_df, X_valid, y_train_df, y_valid = train_test_split(train_df, y, 
#                                                                 test_size=.2, stratify=y_train, 
#                                                                 random_state=17)
    
#     for train_index, valid_index in cv.split(X_train_df, y_train_df):
#         X_tr = X_train_df.iloc[train_index, :]
#         X_val = X_train_df.iloc[valid_index, :]
#         y_tr = y_train_df.iloc[train_index]
#         y_val = y_train_df.iloc[valid_index]
        
#         model = cat.fit(X_tr, y_tr, cat_features=categorical_cols)
        
#         oof_train[valid_index] = model.predict(X_val,
#                                                 num_iteration=model.best_iteration)
        
        

In [18]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'num_leaves': 25,
    'max_depth': 5,
    'objective': 'binary',
    'learning_rate': 0.03
}

categorical_cols = ['cp_type', 'cp_dose']
oof = target.copy()

In [19]:
for target_col in target.columns:
    if target_col != "sig_id":
        _oof, _preds = run_lgbm(target_col)
        oof[target_col] = _oof
        sub[target_col] = _preds

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.00177492	valid_1's binary_logloss: 0.00715027
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00369204	valid_1's binary_logloss: 0.00709912
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.00176093	valid_1's binary_logloss: 0.00545903
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00408872	valid_1's binary_logloss: 0.00541983
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.00173091	valid_1's binary_logloss: 0.00836047
[20]	training's binary_logloss: 0.00117349	valid_1's binary_logloss: 0.00831138
[30]	training's binary_logloss: 0.00083478	valid_1's binary_logloss: 0.00827508
Early stopping, best iteration is:
[26]	training's binary_logloss: 0.000956673	valid_1's binary_logloss: 0.00821336
Training until validation scores don't improve for 10 rounds
[10]	trainin

In [20]:
scores = []
for target_col in target.columns:
    if target_col != "sig_id":
        scores.append(log_loss(target[target_col], oof[target_col]))
print(np.mean(scores))

0.01662072317699001


In [21]:
sub.to_csv('submission.csv', index=False)